## Federated Fairness Analytics - FedAvg
Evaluating fairness for FedAvg on the FEMNIST dataset in the iid setting.

### Imports, Dependencies and Config
Firstly, installing the dependencies and importing what we need.

In [ ]:
!pip install torch torchvision
!pip install tensorflow tensorflow-datasets tensorflow-federated
!pip install -q flwr[simulation]

from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.datasets import EMNIST
from torch.utils.data import DataLoader, random_split
import random
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

from math import comb
from itertools import combinations
from google.colab import drive
import json
from datetime import timedelta
import time
start = time.perf_counter()

import flwr as fl
from flwr.common import Metrics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.48.2 requires protobuf<4.0dev,>=3.12.0, but you have protobuf 4.25.3 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.14.1 which is incompatible.


### Config

In [ ]:
mode = "cuda" # "cuda" to train on GPU
DEVICE = torch.device(mode)
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)
# Parameter Initialisation
NUM_CLIENTS = 205
LOCAL_EPOCHS = 5
NUM_ROUNDS = 20
BATCH_SIZE = 32
SELECTION_RATE = 0.025 # what proportion of clients are selected per round
SENSITIVE_ATTRIBUTES = [0,1,2,3,4,5,6,7,8,9] # a list of the numbers corresponding to sensitive classes
# Preparing data structure for JSON export
data = {
    "rounds": [],
    "general_fairness": {
          "f_j": [],
          "f_g": [],
          "f_r": [],
          "f_o": []},
      "config": {
          "num_clients": NUM_CLIENTS,
          "local_epochs": LOCAL_EPOCHS,
          "num_rounds": NUM_ROUNDS,
          "batch_size": BATCH_SIZE,
          "selection_rate": SELECTION_RATE,
          "sensitive_attributes": SENSITIVE_ATTRIBUTES},
      "per_client_data": {
            "shap": [],
            "accuracy": [],
            "avg_eop": [],
            "gains": []}}
# Setting up for Data Storage:
drive.mount('/content/drive')
path_extension = f'FedAvg_FEMNIST_iid_{NUM_CLIENTS}C_{int(SELECTION_RATE * 100)}PC_{LOCAL_EPOCHS}E_{NUM_ROUNDS}R_v2'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Training on cuda using PyTorch 2.2.1+cu121 and Flower 1.7.0
Mounted at /content/drive


### Data Loading
Beginning with CIFAR-10, loading the data across NUM_CLIENTS. The function load_datasets(...) will partition the database for the number of clients selected. See the following links that are useful for deployment. Using the tensorflow federated EMNIST dataset and converting to torch data objects.

https://flower.ai/docs/datasets/tutorial-quickstart.html

https://www.tensorflow.org/federated/tutorials/loading_remote_data

https://github.com/TalwalkarLab/leaf/tree/master/data/femnist

https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData



In [ ]:
def load_datasets(num_clients: int):
    # Loading and processing the FEMNIST dataset for the non-iid setting.
    # Use this maths to determine the number we need to sample for test set
    # Normal is 671,585: 77,483 train:test for 3400 clients
    # Average number of train per client is 197.525
    # Average number of val per client is 22.789
    # Total number of samples per client is 230.3
    # 205/3400 = 0.0602941, therefore 0.0602941(671585) = 40492.625 train samples
    # 0.0602941(77483) = 4671.7677503 train samples hence 45164.392 total distributed
    # To maintain the same train to test ratios, 45164.392*(77483/671585) = 5210.766 test
    # Number of clients for test = 5210.766 / 22.789 = 228.65 clients from the test to samples
    train_length = 198 # Proportions based on the average division used in the niid case.
    val_length = 23
    test_set_size = int(NUM_CLIENTS*(((NUM_CLIENTS / 3400) * (671585 + 77483) * (77483 / 671585)) / (77483 / 3400)))
    trainset, testset = tff.simulation.datasets.emnist.load_data(only_digits=False, cache_dir=None)
    trainset = list(tfds.as_numpy(testset.create_tf_dataset_from_all_clients()))
    testset = list(tfds.as_numpy(testset.create_tf_dataset_from_all_clients()))
    partition_size = train_length + val_length # to achieve the same average number of samples per client as the niid setting
    lengths = [partition_size] * num_clients
    lengths.append(len(trainset) - int(np.sum(np.array(lengths))))
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42)) # manual seed defines the pseudo random generator
    testset = random_split(testset, [test_set_size, (len(testset) - test_set_size)], torch.Generator().manual_seed(42))
    trainloaders = []
    valloaders = []
    for i in range(len(datasets) - 1):
        ds = datasets[i]
        len_val = len(ds) // (train_length / val_length)  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [int(len_train), int(len_val)]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, drop_last=False))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=True))
    testloader = DataLoader(testset[0], batch_size=BATCH_SIZE, drop_last=False)
    # print(f"Length of trainloaders = {len(trainloaders)}, Length of valloaders = {len(valloaders)}")
    # for i in range(NUM_CLIENTS):
    #     print(f"Client {i} has trainloader length = {len(trainloaders[i])} batches of {NUM_BATCHES}")
    #     print(f"Client {i} has valloader length = {len(valloaders[i])} batches of {NUM_BATCHES}")
    # # Visualising the dataset
    # batch = next(iter(trainloaders[0]))
    # images, labels = batch['pixels'], batch['label']
    emnist_labels = {0: '0',1: '1',2: '2',3: '3',4: '4',5: '5',6: '6',7: '7',8: '8',9: '9',10: 'A',
    11: 'B',12: 'C',13: 'D',14: 'E',15: 'F',16: 'G',17: 'H',18: 'I',19: 'J',20: 'K',21: 'L',22: 'M',
    23: 'N',24: 'O',25: 'P',26: 'Q',27: 'R',28: 'S',29: 'T',30: 'U',31: 'V',32: 'W',33: 'X',34: 'Y',
    35: 'Z',36: 'a',37: 'b',38: 'c',39: 'd',40: 'e',41: 'f',42: 'g',43: 'h',44: 'i',45: 'j',46: 'k',47: 'l',
    48: 'm',49: 'n',50: 'o',51: 'p',52: 'q',53: 'r',54: 's',55: 't',56: 'u',57: 'v',58: 'w',59: 'x',60: 'y',61: 'z'}
    # Use the following to visualise the data:
    # fig, axs = plt.subplots(4,8,figsize=(12,6))
    # print("A trainloader sample:")
    # for i, ax in enumerate(axs.flat):
    #   ax.imshow(images[i], cmap='gray')
    #   ax.set_title(emnist_labels[int(labels[i])])
    #   ax.axis("off")
    # plt.tight_layout()
    # plt.show()
    # # Investigating the test set
    # print("A Testloader sample")
    # batch = next(iter(testloader))
    # images, labels = batch['pixels'], batch['label']
    # fig, axs = plt.subplots(4,8,figsize=(12,6))
    # for i, ax in enumerate(axs.flat):
    #   ax.imshow(images[i], cmap='gray')
    #   ax.set_title(emnist_labels[int(labels[i])])
    #   ax.axis("off")
    # plt.tight_layout()
    # plt.show()
    return trainloaders, valloaders, testloader, emnist_labels.values()

### Training/ Evaluation
We need the get(...) and the set(...) functions for moving model parameters to and from the clients and between Numpy ndarrays which flower can operate on.

Adapted from the basic skeleton of the Flower tutorial: https://flower.ai/docs/framework/tutorial-series-use-a-federated-learning-strategy-pytorch.html

In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__() # Calls init method of Net superclass (nn.Module) enabling access to nn
        self.fmaps1 = 40
        self.fmaps2 = 160
        self.dense = 200
        self.dropout = 0.4
        self.batch_size = 32
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=self.fmaps1, kernel_size=5, stride=1, padding='same'),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=self.fmaps1, out_channels=self.fmaps2, kernel_size=5, stride=1, padding='same'),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.fcon1 = nn.Sequential(nn.Linear(49*self.fmaps2, self.dense), nn.LeakyReLU())
        self.fcon2 = nn.Linear(self.dense, 62)
        self.dropout = nn.Dropout(p=self.dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor: # -> is an annotation for function output
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.fcon1(x))
        x = self.fcon2(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    # taking state_dict values to numpy (state_dict holds learnable parameters)
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    # Setting the new parameters in the state_dict from numpy that flower operated on
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images = batch['pixels']
            labels = batch['label']
            length = len(images)
            images = torch.reshape(images, (length, 1, 28, 28)) # required to meet NN input shape
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels.long())
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += length
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader, sensitive_labels=[]):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    group_performance = [[0,0] for label in range(len(sensitive_labels))] # preset for EOP calc, will store the performance
    # init array for storing EOP information
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images = batch['pixels']
            labels = batch['label']
            length = len(images)
            images = torch.reshape(images, (length, 1, 28, 28)) # required to meet NN input shape
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels.long()).item()
            _, predicted = torch.max(outputs.data, 1)
            # Comparing the predicted to the inputs in order to determine EOP
            matched = (predicted == labels)
            for label in range(len(sensitive_labels)):
              labelled = (labels == label)
              not_labelled = (labels != label)
              group_performance[label][0] += (matched == labelled).sum() # issue is that it will log it correctly if both were wrong
              group_performance[label][1] += (matched == not_labelled).sum()
            total += length
            correct += matched.sum().item()
    for index in range(len(group_performance)):
      # Calculating P(Y.=1|A=1,Y=1) - P(Y.=1|A=0,Y=1) for each:
      # NB: could expand EOP to EOD by accounting for all results not just the correct results, seeing if predictions match
        group_performance[index] = float((group_performance[index][0] - group_performance[index][1]) / total)
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy, group_performance

### Creating the Flower Client
We pass in cid for additional logging.

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # The training method for the client
        # Need to use the config dictionary in order to
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]
        sensitive_attributes = config["sensitive_attributes"]
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        _, reward, _ = test(self.net, self.valloader, []) # The reward is defined as the accuracy of the model on the central data
        train(self.net, self.trainloader, epochs=local_epochs)
        # Performing federated evaluation on the clients that are sampled for training:
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, get_parameters(self.net)) # getting end of training round accuracy
        loss, accuracy, group_eod = test(self.net, self.valloader, sensitive_attributes)
        # Need to process the EOD data here to determine group fairness:
        group_fairness = dict(zip(sensitive_attributes, group_eod))
        # Can include enhanced processing to show which groups are not performing well (eop's further from zero)
        return get_parameters(self.net), len(self.trainloader), {"cid":int(self.cid), "parameters": get_parameters(self.net), "accuracy": float(accuracy), "loss": float(loss), "group_fairness": group_fairness, "reward": float(reward)}


def client_fn(cid) -> FlowerClient:
    # Instances of clients are only created when required to avoid depleting RAM.
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

### Customisation
Using the config dictionary to alter client behaviour.

In [ ]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": LOCAL_EPOCHS,
        "sensitive_attributes": SENSITIVE_ATTRIBUTES,
    }
    return config

### Shapley Values
First attempt at implementing the Shapley calculations.

In [ ]:
class Shapley():
  """
  Implementation of the different types of the Federated Shapley value.
  """
  def __init__(self, test_set, test_function, set_parameters, number_clients: int):
    self.dataset = test_set
    self.test = test
    self.set_parameters = set_parameters
    self.num_clients = number_clients
    self.resultsFedSV = np.zeros(NUM_CLIENTS) # Holding the FedSV Shapley values indexed by client id
    self.aggregatedRoundParams = None # if required we can use: fl.common.ndarrays_to_parameters(get_parameters(Net()))
    self.centralLoss = 0 # Taken from the centralised evaluation as it is not necessary to compute twice
    self.round = 0 # Auto updated by the evaluation function for debugging
    self.f_o = 0
    self.net = Net().to(DEVICE)
    return

  def __utility_function(self, comparitive_weights):#: List[fl.common.NDArrays]):
    """
    Per round utility function.
    Input: array of model weights, if a nested array then average of the set's
           weights obtained.
    Output: the difference in the loss between the aggregated weights for the round
            and the average of the input set of weights.
    """
    # Perform an average on the comparitive weights
    set_size = len(comparitive_weights)
    if set_size == 1:
      alt_weights = comparitive_weights[0]
    else:
      alt_weights = np.mean([np.array(weights, dtype=object) for weights in comparitive_weights], axis=0)
    # Initialising a test net to compare to the centrally evaluated loss.
    self.set_parameters(self.net, list(alt_weights)) # issue with alt_weights averaging
    loss, _, _ = self.test(self.net, self.dataset) # Testing on the central dataset
    return float(self.centralLoss - loss)

  def __power_set(self, input_set):
    """
    Returns the improper power set of the input set, the empty set is removed as
    it does not make sense to train a model on zero weights
    """
    input_list = list(input_set)
    power_set =  [list(c) for r in range(len(input_list) + 1) for c in combinations(input_list, r)]
    return power_set[1:]

  def fedSV(self, round_participants,
            participant_weights: List[fl.common.NDArrays]):
    """
    Calculating the original federated Shapley value.
    Method is called each round when the training has been completed and the
    server has aggregated parameters to form the new model.
    """
    # NOTE - big issues around central dataset assumption - needs to be iid
    # and representative etc - all that I didn't want to have to do
    print(f"Calculating the round {self.round} Shapley values")
    for i in range(self.num_clients):
      # if the client has not participated in training, skip, it is assigned
      # a Shapley value of zero for that round:
      if i not in round_participants:
        continue
      else:
        # find the power set - all the subsets without client i
        contributions = 0
        s_t = 0
        set_without_i = round_participants.copy()
        set_without_i.remove(i)
        #print(f"We find the power set of {round_participants} without {i} which is {set_without_i}")
        power_set = self.__power_set(set_without_i)
        power_weights = [[participant_weights[j] for j in s] for s in power_set]
        for s in power_weights:
          without_i = self.__utility_function(s)
          t = s.copy()
          t.append(participant_weights[i])
          with_i = self.__utility_function(t)
          # We accumulate the contributions from each client for each round:
          contributions += (1 / comb(len(round_participants)-1,len(s))) * (with_i - without_i)
        s_t = contributions / len(round_participants)
        print(f"Client {i} has Shapley contribution {s_t}")
        self.resultsFedSV[i] = self.resultsFedSV[i] + s_t # updating the Shapley value
    return

### Evaluation Metrics
Passing a custom function into the strategy, it will call the function whenever it receives an evaluation metric dictionaries from the client. The evaluate function in the client has performed **federated evaluation** and we use this data to post the average accuracy. The aim is to expand this function and the client evalation method in order to calculate our fairness metrics.

In [ ]:
def evaluate(server_round: int,
             parameters: fl.common.NDArrays,
             config: Dict[str, fl.common.Scalar]) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
             # Used for centralised evaluation. This is enacted by flower before the Federated Evaluation.
             # Runs initially before FL begins as well.
    net = Net().to(DEVICE)
    shap.aggregatedRoundParams = parameters
    set_parameters(net, parameters)
    loss, accuracy, _ = test(net, testloader)
    shap.f_o = accuracy
    shap.centralLoss = loss
    shap.round = server_round
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}


def fit_callback(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Called at the end of the clients fit method
    # Used to call the Shapley calculation as it is before weight aggregation
    clients = set()
    parameters = [None for client in range(NUM_CLIENTS)]
    # Why are the parameters we get here the post aggregation ones...?
    for client in metrics:
      cid = client[1]["cid"]
      clients.add(cid)
      parameters[cid] = client[1]["parameters"]
    if True:
      shap.fedSV(clients, parameters)
    # Jain's fairness index (JFI) is used to evaluate uniformity for the fairness metrics
    JFI = lambda x: ((np.sum(x)**2) / (len(x) * np.sum(x**2)))
    # We determine individual fairness using the FedEval accuracy and JFI
    accuracies = np.array([metric["accuracy"] for _,metric in metrics])
    rewards = np.array([metric["reward"] for _,metric in metrics])
    contributions = shap.resultsFedSV
    gains = np.array([accuracies[i] / contributions[metrics[i][1]["cid"]] for i in range(len(metrics))])
    f_j = JFI(gains)
    print(f"Individual fairness, f_j = {f_j}")
    # As we have passed the sensitive labels into fedEval, we calculate f_g
    group_fairness = np.array([metric["group_fairness"] for _,metric in metrics])
    # Linear mapping the average EOD which is between [-1,1] to [0,1] by taking mod (this is okay as either are unfair just represents difference in false )
    f_g = 1 - np.mean(np.median(np.absolute(np.array([np.mean(np.array(list(group_dict.values()))) for group_dict in group_fairness]))))
    print(f"Group fairness, f_g = {f_g}")
    # We calculate incentive fairness using the reward (accuracy) and contributions
    # We only get accuracies of the evaluated client set, we match the contribution by cid
    reward_gains = np.array([rewards[i] / contributions[metrics[i][1]["cid"]] for i in range(len(metrics))])
    if np.sum(reward_gains) == 0: # Used to catch the case where the accuracy is zero for all clients which would break JFI
      f_r = 1 # if all are zero, it is technically uniform
    else:
      f_r = JFI(reward_gains)
    print(f"Incentive fairness, f_r = {f_r}")
    # Obtain the orchestrator fairness back from the Shapley class
    # f_o = shap.f_o # centralised evaluation option
    f_o = np.mean(accuracies)
    print(f"Orchestrator fairness, f_o = {f_o}")
    # Saving metrics to dictionary for JSON storage:
    data["rounds"].append(shap.round)
    data["general_fairness"]["f_j"].append(f_j)
    data["general_fairness"]["f_g"].append(f_g)
    data["general_fairness"]["f_r"].append(f_r)
    data["general_fairness"]["f_o"].append(f_o)
    data["per_client_data"]["shap"].append(list(contributions))
    data["per_client_data"]["accuracy"].append(list(accuracies))
    data["per_client_data"]["avg_eop"].append(list(group_fairness))
    data["per_client_data"]["gains"].append(list(gains))

    return {"f_j": f_j, "f_g": f_g, "f_r": f_r, "f_o": f_o}

### Strategy Customisation
In many cases, we want more control over parameter initialisation for the global model (which defaults to initialising the global model by asking one random client for the initial parameters). We pass initial parameters directly to the strategy.

Docs for strategies: https://flower.dev/docs/framework/how-to-implement-strategies.html

The strategy defines the approach we are using and it is passed to the start simulation function.

In [ ]:
# Gathering the data:
import pickle
trainloaders, valloaders, testloader, text_labels = load_datasets(NUM_CLIENTS)
data = {"train": trainloaders,
        "val": valloaders,
        "test": testloader,
        "labels": list(text_labels)}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
with open('/content/drive/My Drive/FL/Results/'+ 'femnist_iid_loaded' + '.pickle', "wb") as outfile:
    pickle.dump(data, outfile)

In [ ]:
# Creating Shapley instance
shap = Shapley(testloader, test, set_parameters, NUM_CLIENTS)
# Create FedAvg strategy:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=SELECTION_RATE, # sample all clients for training
    fraction_evaluate=0.0, # Disabling federated evaluation
    min_fit_clients=int(NUM_CLIENTS*SELECTION_RATE), # never sample less that this for training
    min_evaluate_clients=int(NUM_CLIENTS*SELECTION_RATE), # never sample less than this for evaluation
    min_available_clients=NUM_CLIENTS, # has to wait until all clients are available
    # Passing initial_parameters prevents flower asking a client:
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    # Called whenever fit or evaluate metrics are received from clients:
    fit_metrics_aggregation_fn = fit_callback,
    # Evaluate function is called by flower every round for central evaluation:
    evaluate_fn=evaluate,
    # Altering client behaviour with the config dictionary:
    on_fit_config_fn=fit_config,
)

# Specifying client resources
client_resources = None # {"num_cpus": 1, "num_gpus": 0.0}
if DEVICE.type == "cuda":
    # here we are asigning an entire GPU for each client.
    client_resources = {"num_cpus": 1, "num_gpus": 1.0}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),  # We can configure the number of rounds - we want this to have a threshold at which it cuts off
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2024-03-20 10:52:35,086 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2024-03-20 10:52:38,671	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-20 10:52:40,947 | app.py:213 | Flower VCE: Ray initialized with resources: {'accelerator_type:V100': 1.0, 'memory': 7864929486.0, 'object_store_memory': 3932464742.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'CPU': 2.0, 'node:__internal_head__': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'accelerator_type:V100': 1.0, 'memory': 7864929486.0, 'object_store_memory': 3932464742.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'CPU': 2.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-03-20 10:52:40,955 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: 

Server-side evaluation loss 0.12916205187897586 / accuracy 0.005546903334542274


(pid=10387) 2024-03-20 10:52:45.560689: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=10387) 2024-03-20 10:52:45.560761: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=10387) 2024-03-20 10:52:45.560817: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=10387) 2024-03-20 10:52:48.101981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(DefaultActor pid=10387) /usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_actor.py:72: DeprecationWarning:  Ensure your client is of type `flwr.client.Client`. Pleas

(DefaultActor pid=10387) [Client 128, round 1] fit, config: {'server_round': 1, 'local_epochs': 5, 'sensitive_attributes': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
(DefaultActor pid=10387) Epoch 1: train loss 0.14488454163074493, accuracy 0.03571428571428571
(DefaultActor pid=10387) Epoch 2: train loss 0.13532094657421112, accuracy 0.07142857142857142
(DefaultActor pid=10387) Epoch 3: train loss 0.13677901029586792, accuracy 0.07142857142857142
(DefaultActor pid=10387) Epoch 4: train loss 0.1358821988105774, accuracy 0.04081632653061224
(DefaultActor pid=10387) Epoch 5: train loss 0.13094434142112732, accuracy 0.07653061224489796
(DefaultActor pid=10387) [Client 128] evaluate, config: {'server_round': 1, 'local_epochs': 5, 'sensitive_attributes': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


(DefaultActor pid=10387) /usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_actor.py:72: DeprecationWarning:  Ensure your client is of type `flwr.client.Client`. Please convert it using the `.to_client()` method before returning it in the `client_fn` you pass to `start_simulation`. We have applied this conversion on your behalf. Not returning a `Client` might trigger an error in future versions of Flower.
(DefaultActor pid=10387)   client = check_clientfn_returns_client(client_fn(cid))


...

DEBUG flwr 2024-03-20 12:25:48,981 | server.py:236 | fit_round 20 received 5 results and 0 failures
DEBUG:flwr:fit_round 20 received 5 results and 0 failures


(DefaultActor pid=10387) [Client 50, round 20] fit, config: {'server_round': 20, 'local_epochs': 5, 'sensitive_attributes': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
(DefaultActor pid=10387) Epoch 1: train loss 0.03611742705106735, accuracy 0.7040816326530612
(DefaultActor pid=10387) Epoch 2: train loss 0.02962704934179783, accuracy 0.7653061224489796
(DefaultActor pid=10387) Epoch 3: train loss 0.020048851147294044, accuracy 0.826530612244898
(DefaultActor pid=10387) Epoch 4: train loss 0.017984537407755852, accuracy 0.8775510204081632
Calculating the round 19 Shapley values
(DefaultActor pid=10387) Epoch 5: train loss 0.0118995551019907, accuracy 0.9132653061224489
(DefaultActor pid=10387) [Client 50] evaluate, config: {'server_round': 20, 'local_epochs': 5, 'sensitive_attributes': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
Client 4 has Shapley contribution 0.002148629615710153
Client 50 has Shapley contribution 0.0019857436125636846
Client 54 has Shapley contribution 0.0012243616894438663
Client 112 has

INFO flwr 2024-03-20 12:30:36,420 | server.py:125 | fit progress: (20, 0.030459371605127062, {'accuracy': 0.7473172188680051}, 5871.093143730999)
INFO:flwr:fit progress: (20, 0.030459371605127062, {'accuracy': 0.7473172188680051}, 5871.093143730999)
INFO flwr 2024-03-20 12:30:36,429 | server.py:171 | evaluate_round 20: no clients selected, cancel
INFO:flwr:evaluate_round 20: no clients selected, cancel
INFO flwr 2024-03-20 12:30:36,432 | server.py:153 | FL finished in 5871.105456963
INFO:flwr:FL finished in 5871.105456963
INFO flwr 2024-03-20 12:30:36,441 | app.py:226 | app_fit: losses_distributed []
INFO:flwr:app_fit: losses_distributed []
INFO flwr 2024-03-20 12:30:36,443 | app.py:227 | app_fit: metrics_distributed_fit {'f_j': [(1, 0.7599797810426164), (2, 0.3328334485251469), (3, 0.11787373966839894), (4, 0.43974446718211796), (5, 0.5659404628765677), (6, 0.8516084974260238), (7, 0.12995805364329807), (8, 0.9342112648744338), (9, 0.7829044423911735), (10, 0.8717665710944317), (11, 0

Server-side evaluation loss 0.030459371605127062 / accuracy 0.7473172188680051


History (loss, centralized):
	round 0: 0.12916205187897586
	round 1: 0.12440716528799435
	round 2: 0.11417746463517893
	round 3: 0.10189816944783231
	round 4: 0.0852109406238719
	round 5: 0.07075520375400926
	round 6: 0.05819377410638946
	round 7: 0.054700535158014076
	round 8: 0.04880917371635493
	round 9: 0.043506447428553016
	round 10: 0.04056903966977807
	round 11: 0.0388432933192499
	round 12: 0.03668699396609538
	round 13: 0.035100623289079425
	round 14: 0.03447514242467522
	round 15: 0.0331225147897977
	round 16: 0.032545702634577366
	round 17: 0.03124842846014388
	round 18: 0.0296954416426688
	round 19: 0.02995062164064687
	round 20: 0.030459371605127062
History (metrics, distributed, fit):
{'f_j': [(1, 0.7599797810426164), (2, 0.3328334485251469), (3, 0.11787373966839894), (4, 0.43974446718211796), (5, 0.5659404628765677), (6, 0.8516084974260238), (7, 0.12995805364329807), (8, 0.9342112648744338), (9, 0.7829044423911735), (10, 0.8717665710944317), (11, 0.9534105822789635), (12

In [ ]:
# Saving data:
with open('/content/drive/My Drive/FL/Results/'+ path_extension + '.json', "w") as outfile:
  data = json.dump(data, outfile)
print(f"Elapsed time in {mode} mode = {timedelta(seconds=time.perf_counter()-start)}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Elapsed time in cuda mode = 1:48:04.107980
